# 311 Calls per NC

Simple idioms I use to display with ipyleaflet.

In this notebook we'll use one of the attributes of the 311 data (nc) to focus on the map.

Steps:

1.  Initial imports
2.  Read data
3.  Explore the data
4.  Pick an nc to view
5.  Build the layers
6.  Render on the map

# 1. Setup

I'm assuming you have an environment ala the environment.yml file.  

I have a startup script in the ~/.ipython/profile_default/startup directory.  I've copied it to this folder so you can run it for the same effect.

**Note:** I'm importing the utility function read_new311_shape to read the preproccessed 311 data and map the column names/types.

In [1]:
%run start.py
import nc

from utils import read_new311_shape

# 2. Get the Data

In [2]:
%%time
new311_gdf = read_new311_shape('../data/311/new311-shape.zip')

CPU times: user 1min 33s, sys: 2.36 s, total: 1min 35s
Wall time: 1min 35s


In [3]:
ncs_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.zip/')

In [4]:
ncs_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

# 3. Examine the Data

In [5]:
new311_gdf.columns

Index(['SRNumber', 'created_dt', 'updated_dt', 'owner', 'request_type',
       'service_dt', 'closed_dt', 'address', 'street', 'zip_code', 'latitude',
       'longitude', 'location', 'APC', 'cd', 'cd_member', 'nc', 'nc_name',
       'precinct', 'days_to_service', 'days_to_close', 'days_to_update',
       'geometry'],
      dtype='object')

In [6]:
new311_gdf.request_type.value_counts()

Bulky Items                   629583
Graffiti Removal              315577
Metal/Household Appliances    114004
Illegal Dumping Pickup        106324
Homeless Encampment            43250
Electronic Waste               37464
Other                          25430
Dead Animal Removal            23298
Single Streetlight Issue       10678
Multiple Streetlight Issue      8544
Report Water Waste              1558
Feedback                         270
Name: request_type, dtype: int64

In [7]:
new311_gdf.nc_name.value_counts().to_frame().head()

,nc_name
South Central,39367
Boyle Heights,37591
Empowerment Congress Southeast,33647
Sylmar,29301
Wilshire Center Koreatown,26422


In [8]:
small_gdf = new311_gdf.query(f"nc == 46").reset_index().drop(columns=['index'])
#small_gdf = new311_gdf.query(f"nc == 79").reset_index().drop(columns=['index'])

In [9]:
small_gdf.request_type.value_counts()

Graffiti Removal              1156
Bulky Items                    248
Homeless Encampment            133
Other                           62
Illegal Dumping Pickup          55
Metal/Household Appliances      45
Dead Animal Removal             23
Multiple Streetlight Issue      14
Electronic Waste                11
Single Streetlight Issue        11
Feedback                         2
Name: request_type, dtype: int64

In [10]:
small_gdf.created_dt.min()

Timestamp('2021-01-01 07:47:18')

In [11]:
small_gdf.created_dt.max()

Timestamp('2021-11-19 13:13:37')

In [12]:
(_ - __).days / 7

46.0

In [13]:
keys = new311_gdf.request_type.value_counts().keys().to_list()

vals = [
    '#267370',
    '#8B508B',
    '#EDAD08',
    '#1D6996',
    '#11975F',
    '#E17C05',
    '#685DB1',
    '#AE3D51',
    '#79B74E',
    '#BF82BA',
    '#D05F4E',
    '#33BBEE'
]

c_map = {kv[0]: kv[1] for kv in zip(keys, vals)}

c_map

{'Bulky Items': '#267370',
 'Graffiti Removal': '#8B508B',
 'Metal/Household Appliances': '#EDAD08',
 'Illegal Dumping Pickup': '#1D6996',
 'Homeless Encampment': '#11975F',
 'Electronic Waste': '#E17C05',
 'Other': '#685DB1',
 'Dead Animal Removal': '#AE3D51',
 'Single Streetlight Issue': '#79B74E',
 'Multiple Streetlight Issue': '#BF82BA',
 'Report Water Waste': '#D05F4E',
 'Feedback': '#33BBEE'}

In [14]:
markers = list()

for i, row in tqdm(small_gdf.iterrows()):
    
    fill_color = c_map[row.request_type]
    marker = CircleMarker(location=(row.geometry.y, row.geometry.x), radius=5, stroke=False, fill_color=fill_color, fill_opacity=1.0)
    msg = HTML()
    msg.value = "report: {}<br>Address: {}</br>when: {}<br/>type: {}".format(row['SRNumber'], 
                                                              row['address'],
                                                            row['created_dt'].strftime("%m/%d/%Y, %H:%M"),
                                                            row['request_type']) #"status: {}<br/>coord: {}".format(r['status'], r['coordinates'])
    marker.popup = msg
    markers.append(marker)
    small_gdf.loc[i, 'marker'] = marker

little_tokyo_311_cluster = MarkerCluster(markers=markers, name='311 Calls')


1760it [00:10, 172.69it/s]


In [15]:
#ncs_gdf.to_crs(epsg=4326, inplace=True)
little_tokyo_gdf = ncs_gdf.query(f"nc_id == 46").reset_index()

In [16]:
center = little_tokyo_gdf.iloc[0].geometry.centroid.y, little_tokyo_gdf.iloc[0].geometry.centroid.x

In [17]:
nc_layer = GeoData(geo_dataframe = little_tokyo_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.8, 'weight':1.9, 'dashArray':'5', 'fillOpacity':0.2},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Little Tokyo NC')

In [18]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True

from ipyleaflet import TileLayer
google_map = TileLayer(
    url="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Maps",
)
google_map.base = True

google_satellite = TileLayer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Satellite"
)
google_satellite.base = True

map_display = Map(center=center, zoom=15,
                  layers=[google_satellite, google_map, imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())

#map_display.add_layer(geo_data)
map_display += nc_layer
map_display.add_layer(little_tokyo_311_cluster)

all_311 = LayerGroup(name=f"All 311", layers=markers)
map_display += all_311
map_display

Map(center=[34.04480040479194, -118.2349337043463], controls=(ZoomControl(options=['position', 'zoom_in_text',…